Reference: 
https://github.com/AlexanderNixon/Machine-learning-reads/blob/master/Movie-content-based-recommender-using-tf-idf.ipynb
https://ithelp.ithome.com.tw/articles/10220479

In [427]:
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import jieba

In [428]:
users_df = pd.read_csv('./data/users.csv')
subgroups_df = pd.read_csv('./data/subgroups.csv')
course_chapter_items_df = pd.read_csv('./data/course_chapter_items.csv')
course_df_ori = pd.read_csv('./data/courses.csv').fillna('')

train_group_df = pd.read_csv('./data/train_group.csv')
test_seen_group_df = pd.read_csv('./data/test_seen_group.csv')
val_seen_group_df = pd.read_csv('./data/val_seen_group.csv')
test_unseen_group_df = pd.read_csv('./data/test_unseen_group.csv')
val_unseen_group_df = pd.read_csv('./data/val_unseen_group.csv')

train_df = pd.read_csv('./data/train.csv')
test_seen_df = pd.read_csv('./data/test_seen.csv')
val_seen_df = pd.read_csv('./data/val_seen.csv')
test_unseen_df = pd.read_csv('./data/test_unseen.csv')
val_unseen_df = pd.read_csv('./data/val_unseen.csv')

In [429]:
id2course_mapping = course_df_ori["course_id"].to_dict()
course2id_mapping = {v : k for k, v in id2course_mapping.items()}
len(course2id_mapping)

728

In [430]:
course_df = course_df_ori.drop(['teacher_id', 'course_published_at_local', 'course_published_at_local', 'course_price'], axis=1)
for i in range(len(course_df)):
    s = ''
    s = course_df['description'][i]
    s1 = BeautifulSoup(s ,'html').text
    course_df['description'][i] = s1

In [431]:
course_df = course_df.replace('\n', '',regex=True).replace('&.;', '',regex=True).replace("--&?", "",regex=True).replace("\t", "",regex=True)

In [432]:
course_df.head(0)

,course_id,course_name,teacher_intro,groups,sub_groups,groups+subgroups,topics,description,will_learn,required_tools,recommended_background,target_group


In [433]:
def clean_data(x):
        return x.replace(" ", "")

fillna=course_df.fillna('')
features= ['course_name', 'teacher_intro', 'groups', 'sub_groups', 'groups+subgroups', 'topics', 'will_learn', 'target_group', 'description', 'recommended_background', 'required_tools']
fillna=fillna[features]
for feature in features:
    fillna[feature] = fillna[feature].apply(clean_data)
    
fillna.head(2)

,course_name,teacher_intro,groups,sub_groups,groups+subgroups,topics,will_learn,target_group,description,recommended_background,required_tools
0,少女人妻華麗變身：七大妝容七彩的夢幻樂園,在美妝KOL圈裡屬個人風格強烈的Alice，在清新與叛逆風格間遊刃有餘，其幽默的美妝影片手法...,生活品味,"更多生活品味,護膚保養與化妝","生活品味_更多生活品味,生活品味_護膚保養與化妝","更多生活品味,護膚保養與化妝",不再害怕各種顏色的彩妝，可以更隨心搭配各種繽紛的顏色。,熱愛彩妝的人,少女人妻第一堂線上課程，跟著我一起華麗變身！在社群上許多人經常問我：「為什麼人妻畫的妝那麼好...,只要你有一顆愛化妝、想變漂亮的心皆可以參加。⚠️雖然課程當中會帶到相關彩妝技巧，不過內容偏向...,所需工具為：視課程實際會用到的彩妝用品
1,幾何圖形分割X色塊組合,從學生時代開始摸索photoshop等軟體，自以為有些天賦但後來發現其實沒有。出社會後從事美...,"藝術,設計","平面設計,繪畫與插畫","設計_平面設計,藝術_繪畫與插畫","Illustrator/以拉,配色技巧",可以將任何區塊分割成自己想要的幾何圖形、快速的上色，並且能夠應用在許多地方,每一位興趣的人都能學得來，非常容易的小技巧,"這是一個有趣的小技巧，主要利用illustrator裡面的""分割“這個功能，我已經想不起來是...",知道如何使用Illustrator的基本工具列,"AdobeIllustrator（必備）,camera"


In [434]:
from chinese_keybert import Chinese_Extractor
kw_extractor = Chinese_Extractor()

In [435]:
fillna['kw_name'] = ''
fillna['kw_intro'] = ''
fillna['kw_topics'] = ''
fillna['kw_will'] = ''
fillna['kw_target'] = ''
fillna['kw_desc'] = ''
fillna['kw_recommend'] = ''
fillna['kw_tool'] = ''

In [436]:
for i in range(len(fillna)):
    s1_name = ''
    s1_intro = ''
    s1_topics = ''
    s1_will = ''
    s1_target = ''
    s1_desc = ''
    s1_recommend = ''
    s1_tool = ''
    s_name = fillna['course_name'][i]
    s_intro = fillna['teacher_intro'][i]
    s_topics = fillna['topics'][i]
    s_will = fillna['will_learn'][i]
    s_target = fillna['target_group'][i]
    s_desc = fillna['description'][i]
    s_recommend = fillna['recommended_background'][i]
    s_tool = fillna['required_tools'][i]
    print(i)
    try:
        try:
            kw_name = kw_extractor.generate_keywords(s_name,top_k=3,rank_methods="mmr",diversity=0.15)
            for j in range(3):
                s1_name = s1_name + kw_name[0][j] + ','
            fillna['kw_name'][i] = s1_name
        except:
            continue
        try:
            kw_intro = kw_extractor.generate_keywords(s_intro,top_k=5,rank_methods="mmr",diversity=0.15)
            for k in range(5):
                s1_intro = s1_intro + kw_intro[0][k] + ','
            fillna['kw_intro'][i] = s1_intro
        except:
            continue
        try:
            kw_topics = kw_extractor.generate_keywords(s_topics,top_k=3,rank_methods="mmr",diversity=0.15)
            for j in range(3):
                s1_topics = s1_topics + kw_topics[0][j] + ','
            fillna['kw_topics'][i] = s1_topics
        except:
            continue
        try:
            kw_will = kw_extractor.generate_keywords(s_will,top_k=3,rank_methods="mmr",diversity=0.15)
            for j in range(3):
                s1_will = s1_will + kw_will[0][j] + ','
            fillna['kw_will'][i] = s1_will
        except:
            continue
        try:
            kw_target = kw_extractor.generate_keywords(s_target,top_k=2,rank_methods="mmr",diversity=0.1)
            s1_target = s1_target + kw_target[0][0] + ','
            fillna['kw_target'][i] = s1_target
        except:
            continue
        try:
            kw_desc = kw_extractor.generate_keywords(s_desc,top_k=5,rank_methods="mmr",diversity=0.15)
            for k in range(5):
                s1_desc = s1_desc + kw_desc[0][k] + ','
            fillna['kw_desc'][i] = s1_desc
        except:
            continue
        try:
            kw_recommend = kw_extractor.generate_keywords(s_recommend,top_k=2,rank_methods="mmr",diversity=0.15)
            for l in range(2):
                s1_recommend = s1_recommend + kw_recommend[0][l] + ','
            fillna['kw_recommend'][i] = s1_recommend
        except:
            continue
        try:
            kw_tool = kw_extractor.generate_keywords(s_tool,top_k=3,rank_methods="mmr",diversity=0.15)
            for l in range(2):
                s1_tool = s1_tool + kw_tool[0][l] + ','
            fillna['kw_tool'][i] = s1_tool
        except:
            continue
        #print(s1)   
    except:
        continue
fillna['kw_name'][0]

0


Inference: 100%|██████████| 1/1 [00:00<00:00, 53.26it/s]


1


Inference: 100%|██████████| 1/1 [00:00<00:00, 21.56it/s]


2


...

Inference: 100%|██████████| 1/1 [00:00<00:00, 43.00it/s]


78


Inference: 100%|██████████| 1/1 [00:00<00:00, 34.40it/s]


79


Inference:   0%|          | 0/1 [00:00<?, ?it/s]

In [383]:
fillna['kw_recommend'][145]

'發音,基礎,初學,英文,改善,'

In [412]:
def create_combination(x):
    return x['course_name']+ ' ' + x['teacher_intro']+ ' ' + x['groups'] + ' ' + x['sub_groups'] +' ' + x['groups+subgroups'] + ' ' + x['topics'] + ' ' + x['will_learn'] + ' ' + x['target_group'] + ' ' + x['description'] + ' ' + x['recommended_background']
def create_combination1(x):
    course_name = x['course_name']
    teacher_intro = x['teacher_intro']
    will_learn = x['will_learn']
    recommended_background = x['recommended_background']
    target_group = x['target_group']
    required_tools = x['required_tools']
    group = x['groups']
    #text = course_name + ' ' + teacher_intro +  ' ' +  group + ' ' + group + ' ' + group + ' ' + x['sub_groups'] +' ' + x['sub_groups'] + ' ' +x['sub_groups'] + ' ' +  x['topics'] + ' ' + will_learn + ' ' + x['groups+subgroups'] + ' ' + teacher_intro
    text = x['kw_target'] +' ' + x['kw_target'] +' ' + x['kw_tool'] +' ' + x['kw_tool'] +' ' + x['kw_intro'] +' ' + x['kw_name'] +' ' + x['kw_name'] +' ' + x['kw_name'] + ' ' + x['kw_desc']+ ' ' + x['kw_desc']+ ' ' + x['teacher_intro']+ ' ' + x['groups'] + ' ' + x['target_group'] + ' ' + x['recommended_background'] + ' ' +x['recommended_background'] + ' ' + x['groups+subgroups']
    #text = course_name + ' ' + teacher_intro + ' ' + x['groups'] + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + recommended_background + ' ' + target_group + ' ' + x['groups+subgroups']

    return text
fillna['combination'] = fillna.apply(create_combination1, axis=1)

In [413]:
fillna.head(2)

,course_name,teacher_intro,groups,sub_groups,groups+subgroups,topics,will_learn,target_group,description,recommended_background,required_tools,kw_desc,combination,combination2,kw_name,kw_intro,kw_recommend,kw_tool,kw_gs,kw_target
0,少女人妻華麗變身：七大妝容七彩的夢幻樂園,在美妝KOL圈裡屬個人風格強烈的Alice，在清新與叛逆風格間遊刃有餘，其幽默的美妝影片手法...,生活品味,"更多生活品味,護膚保養與化妝","生活品味_更多生活品味,生活品味_護膚保養與化妝","更多生活品味,護膚保養與化妝",,熱愛彩妝的人,少女人妻第一堂線上課程，跟著我一起華麗變身！在社群上許多人經常問我：「為什麼人妻畫的妝那麼好...,只要你有一顆愛化妝、想變漂亮的心皆可以參加。⚠️雖然課程當中會帶到相關彩妝技巧，不過內容偏向...,所需工具為：視課程實際會用到的彩妝用品,"彩妝,少女人妻,美妝,畫出,眼妝,","彩妝,課程,工具, 彩妝,課程,工具, 美妝,幽默,粉絲,風格, 妝容,夢幻,七彩, 妝...",在美妝KOL圈裡屬個人風格強烈的Alice，在清新與叛逆風格間遊刃有餘，其幽默的美妝影片手法...,"妝容,夢幻,七彩,","美妝,幽默,粉絲,風格,","彩妝,參加,課程,色系,漂亮,","彩妝,課程,工具,","生活,保養,品味,",
1,幾何圖形分割X色塊組合,從學生時代開始摸索photoshop等軟體，自以為有些天賦但後來發現其實沒有。出社會後從事美...,"藝術,設計","平面設計,繪畫與插畫","設計_平面設計,藝術_繪畫與插畫","Illustrator/以拉,配色技巧",,每一位興趣的人都能學得來，非常容易的小技巧,"這是一個有趣的小技巧，主要利用illustrator裡面的""分割“這個功能，我已經想不起來是...",知道如何使用Illustrator的基本工具列,"AdobeIllustrator（必備）,camera","畫畫,分割,技巧,油漆桶,幾何,","畫畫,設計,印刷,美術, 幾何,分割,圖形, 幾何,分割,圖形, 幾何,分割,圖形,...",從學生時代開始摸索photoshop等軟體，自以為有些天賦但後來發現其實沒有。出社會後從事美...,"幾何,分割,圖形,","畫畫,設計,印刷,美術,",,,"繪畫,設計,藝術,",


In [414]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [415]:
def tokenize(sentence):
    #without_duplicates = re.sub(r'(.)\1+', r'\1\1', sentence)
    without_duplicates = sentence
    bd='*_▌~�❗\'⃣✅▐➤✨❤▒▍–∣⭐'
    for i in bd:
        without_duplicates=without_duplicates.replace(i,'')
    #ws_token = ws(without_duplicates)
    return jieba.lcut(without_duplicates)

In [416]:
tfidf = TfidfVectorizer(tokenizer=jieba.lcut, stop_words='english')
#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(fillna['combination'])
tfidf_matrix_nd = tfidf_matrix.toarray()


In [417]:
def create_combination2(x):
    course_name = x['course_name']
    teacher_intro = x['teacher_intro']
    will_learn = x['will_learn']
    recommended_background = x['recommended_background']
    target_group = x['target_group']
    required_tools = x['required_tools']
    group = x['groups']
    text = teacher_intro +  ' ' +  group + ' ' + group +' ' + x['sub_groups'] + ' ' +x['sub_groups'] + ' ' +  x['topics'] + ' ' + x['groups+subgroups'] + ' ' + teacher_intro  + ' ' + x['groups+subgroups'] + ' ' + x['groups+subgroups'] + ' ' + x['groups+subgroups']
    #text = course_name + ' ' + teacher_intro + ' ' + x['groups'] + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + recommended_background + ' ' + target_group + ' ' + x['groups+subgroups']

    return text
fillna['combination2'] = fillna.apply(create_combination2, axis=1)


In [418]:
tfidf2 = TfidfVectorizer(tokenizer=jieba.lcut, stop_words='english')
#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix2 = tfidf2.fit_transform(fillna['combination2'])
tfidf_matrix_nd2 = tfidf_matrix2.toarray()
np.save('tfidf_embedding/item_embeddings_jieba_english_course1.npy', tfidf_matrix_nd2)

In [419]:
from scipy.sparse import csr_matrix
item_embed_FM = np.load('tfidf_embedding/item_embeddings_jieba_english_course1.npy')
item_embed_FM = item_embed_FM[:728]
print(tfidf_matrix.shape)
print(item_embed_FM.shape)

(728, 15748)
(728, 8630)


In [420]:
concatenate_item_embed_nd = np.concatenate([tfidf_matrix_nd, item_embed_FM/2], axis = 1)
print(concatenate_item_embed_nd.shape)
concatenate_item_embed = csr_matrix(concatenate_item_embed_nd)

(728, 24378)


In [421]:
# Compute the cosine similarity matrix
print(concatenate_item_embed_nd.shape)
cosine_sim2 = linear_kernel(concatenate_item_embed, concatenate_item_embed)
#Output the shape of tfidf_matrix
cosine_sim2.shape

(728, 24378)


(728, 728)

In [422]:
def get_recommendations_new(haved_courses_list, cosine_sim, top = 50):
    cosine_sim_sum = [ [i, 0] for i in range(len(course2id_mapping))]
    haved_courses_index_list = [ course2id_mapping[course_id] for course_id in haved_courses_list]
    for idx in haved_courses_index_list:
        # Get the pairwsie similarity scores of all courses with that course
        sim_scores = list(enumerate(cosine_sim[idx]))
        for i in range(len(sim_scores)):
            cosine_sim_sum[i][1] += sim_scores[i][1]

    # Sort the movies based on the similarity scores
    sim_scores = sorted(cosine_sim_sum, key=lambda x: x[1], reverse=True)

    recommend_indices = []
    # Get the scores of the 10 most course not buy
    for i in range(len(sim_scores)):
        if len(recommend_indices) < top:
            if sim_scores[i][0] not in haved_courses_index_list:
                recommend_indices.append(sim_scores[i][0])

    # Return the top 10 most similar movies
    return course_df['course_id'].iloc[recommend_indices].tolist()
    
get_recommendations_new(['5f194354cad0d086f3ee24cf', '6156a77fdf426a0007cc5fe1'], cosine_sim2, 10)

['5d5aac6e8e9eb60021211902',
 '6184efc3b2319400078aefe7',
 '56a1e0164ec4c609007d2cb0',
 '5f7c210b1de7982fb413a3e9',
 '5fe5f2025a899784aa96f52b',
 '5f7c209762ad22756c7a1c74',
 '5f7c212262ad2203e77a1cc9',
 '6155cda6d425f500065f5c96',
 '5ef616f7303ca82d6d08e2eb',
 '6125b83cdf147200070db995']

In [423]:
seen_predict_df = pd.read_csv('./data/test_seen_group.csv')
predict_user = seen_predict_df["user_id"].to_list()
print(len(predict_user))


seen_user_haved_purchased_course = {}
for seen_user_id, course_ids in zip(train_df["user_id"], train_df["course_id"]):
    course_id_list = course_ids.split(' ')
    seen_user_haved_purchased_course[seen_user_id] = seen_user_haved_purchased_course.setdefault(seen_user_id, []) + course_id_list
for seen_user_id, course_ids in zip(val_seen_df["user_id"], val_seen_df["course_id"]):
    course_id_list = course_ids.split(' ')
    seen_user_haved_purchased_course[seen_user_id] = seen_user_haved_purchased_course.setdefault(seen_user_id, []) + course_id_list
print(len(seen_user_haved_purchased_course))

7205
59737


In [424]:
import csv

predict_users = seen_predict_df["user_id"].to_list()
with open("predict50_82434.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["user_id", "course_id"])
    for user_id in predict_users:
        recommend = " ".join(get_recommendations_new(seen_user_haved_purchased_course[user_id], cosine_sim2, top = 50))
        writer.writerow([user_id, recommend])

# Map50

In [425]:
train_user_haved_purchased_course = {}
for seen_user_id, course_ids in zip(train_df["user_id"], train_df["course_id"]):
    course_id_list = course_ids.split(' ')
    train_user_haved_purchased_course[seen_user_id] = train_user_haved_purchased_course.setdefault(seen_user_id, []) + course_id_list

val_answer_course = {}
val_seen_df_fillna = val_seen_df.fillna("")
for seen_user_id, course_ids in zip(val_seen_df_fillna["user_id"], val_seen_df_fillna["course_id"]):
    if len(course_ids) > 0:
        course_id_list = [ str(x) for x in course_ids.split(' ')]
        val_answer_course[seen_user_id] = val_answer_course.setdefault(seen_user_id, []) + course_id_list
list(val_answer_course.items())[:5]

[('56dae2b74e3ef90900b7bd0e', ['5b61928a8011d1001e356102']),
 ('60e66f29be3e3b0006c4db75', ['559e49185850311000fca504']),
 ('5c919efb728ddf00208b9b2b',
  ['60ddc3ca06259d00064c7f17',
   '60aeac37bca91777bf5bb114',
   '60c84de9eb75ca46e0c25e85',
   '611f7d91bd122100071f2926',
   '611f5d074b76af0007c24d7e',
   '6155cda6d425f500065f5c96',
   '6156a77fdf426a0007cc5fe1']),
 ('5ac115507997a2001e7c3617', ['58d5c70c27ea7d070060160e']),
 ('5f53b84440c5be3bb873a9d3', ['5f55fb39b34335d28416bd0c'])]

In [426]:
import ml_metrics

answers = []
predictions = []
map50s = []
for user_id in val_seen_df["user_id"]:
    prediction = get_recommendations_new(train_user_haved_purchased_course[user_id], cosine_sim2, top = 50)
    #print(prediction)
    prediction_idxs = prediction
    if user_id in val_answer_course.keys():
        answer_idxs = val_answer_course[user_id]
    else:
        answer_idxs = []
        
    # print(user_id)
    # print([course2subgroups[x] for x in train_user_haved_purchased_course[user_id]])
    # print(prediction) 
    # print(answer_idxs)
    # print("")

    predictions.append(prediction_idxs)
    answers.append(answer_idxs)
    map50s.append(ml_metrics.mapk(predicted= [prediction_idxs], actual= [answer_idxs], k = 50))
    
print(np.mean(map50s))
#answers.append(answers)        

map50 = ml_metrics.mapk(predicted= predictions, actual= answers, k = 50)
map50

0.0824505022377611


0.0824505022377611

x['course_name']+ ' ' + x['teacher_intro'] + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + x['will_learn'] + ' ' + x['target_group'] 
0.06871480941842756
x['course_name']+ ' ' + x['teacher_intro']+ ' ' + x['groups'] + ' ' + x['sub_groups'] +' ' + x['groups+subgroups'] + ' ' + x['topics'] + ' ' + x['will_learn'] + ' ' + x['target_group'] + ' ' + x['description'] + ' ' + x['recommended_background']
0.05496563647510346
x['course_name']+ ' ' + x['teacher_intro']+ ' ' + x['groups'] + ' ' + x['sub_groups'] +' ' + x['groups+subgroups'] + ' ' + x['topics'] + ' ' + x['will_learn'] + ' ' + x['target_group'] + ' ' + x['recommended_background']
0.06712582130333207

x['course_name']+ ' ' + x['teacher_intro']+ ' ' + x['groups'] + ' ' + x['sub_groups'] +' ' + x['groups+subgroups'] + ' ' + x['topics'] + ' ' + x['will_learn'] + ' ' + x['target_group']
0.06597312470076507

x['course_name']+ ' ' + x['groups'] + ' ' + x['sub_groups'] +' ' + x['groups+subgroups'] + ' ' + x['topics'] + ' ' + x['will_learn'] + ' ' + x['target_group'] + ' ' + x['recommended_background']
0.03404063813630374

x['course_name']+ ' ' + x['teacher_intro']+ ' ' + x['groups'] + ' ' + x['sub_groups'] +' ' + x['groups+subgroups'] + ' ' + x['topics'] + ' ' + x['target_group'] + ' ' + x['recommended_background']
0.0676415508550833

x['course_name']+ ' ' + x['teacher_intro']+ ' ' + x['groups'] + ' ' + x['sub_groups'] +' ' + x['groups+subgroups'] + ' ' + x['target_group'] + ' ' + x['recommended_background']
0.06939486811195406

x['course_name']+ ' ' + x['teacher_intro']+ ' ' + x['groups'] + ' ' + x['sub_groups'] +' ' + x['target_group'] + ' ' + x['recommended_background']
0.06958664864078341

x['course_name']+ ' ' + x['teacher_intro']+ ' ' + x['sub_groups'] +' ' + x['target_group'] + ' ' + x['recommended_background']
0.06779119882844009
x['course_name']+ ' ' + x['teacher_intro']+ ' ' + x['groups'] + ' ' + x['sub_groups'] +' ' + x['recommended_background']
0.06682865004279648

x['course_name']+ ' ' + x['teacher_intro']+ ' ' + x['groups'] + ' ' + x['sub_groups'] +' ' + x['target_group'] + ' ' + x['recommended_background'] + ' ' + x['sub_groups']
0.06676136192484118

x['course_name']+ ' ' + x['teacher_intro']+ ' ' + x['groups'] + ' ' + x['target_group'] + ' ' + x['recommended_background']
0.06576964508987501

x['teacher_intro'] + ' ' + x['groups'] + ' ' + x['sub_groups'] +' ' + x['target_group'] + ' ' + x['recommended_background']
0.06960659471092882
x['teacher_intro'] + ' ' + x['teacher_intro']+ ' ' + x['groups'] + ' ' + x['sub_groups'] +' ' + x['target_group'] + ' ' + x['recommended_background']
0.06627308178826925
x['teacher_intro'] + ' ' + x['groups'] + ' ' + x['sub_groups'] +' ' + x['target_group'] + ' ' + x['recommended_background']+ ' ' + x['groups']
0.06754423548672533
x['teacher_intro']+ ' ' + x['groups'] + ' ' + x['sub_groups'] +' ' + x['groups+subgroups'] + ' ' + x['target_group'] + ' ' + x['recommended_background']
0.06942449805121588